In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
beer_card.rank, beer_card.suit

('7', 'diamonds')

In [4]:
beer_card[0], beer_card[1]

('7', 'diamonds')

In [5]:
def card_to_str(card):
    return '%s of %s' % card

card_to_str(beer_card)

'7 of diamonds'

In [6]:
Card.__str__ = card_to_str
print(beer_card)

7 of diamonds


In [7]:
deck = FrenchDeck()
len(deck)

52

In [8]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [9]:
Card('Q', 'hearts') in deck

True

In [10]:
Card('Z', 'clubs') in deck

False

In [11]:
from random import choice
choice(deck)

Card(rank='3', suit='diamonds')

In [12]:
for i, card in enumerate(deck):
    print(card)
    if i > 5:
        break

2 of spades
3 of spades
4 of spades
5 of spades
6 of spades
7 of spades
8 of spades


In [13]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high_ordering(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [14]:
spades_high_ordering(Card('2', 'clubs'))

0

In [15]:
spades_high_ordering(Card('A', 'spades'))

51

In [16]:
for card in sorted(deck, key=spades_high):
    print(card)

NameError: name 'spades_high' is not defined

In [ ]:
def setcard(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = setcard

In [ ]:
from random import shuffle

shuffle(deck)
for card in deck:
    print(card)